# Classificazione di vini attraverso gaussiane unidimensionali

<img src="img/wine.jpg" width="30%"/>

Il dataset **Wine** sarà il nostro esempio ricorrente per la discussione dell' *approccio generativo alla classificazione*. 

Il dataset (costruito nel 1991 dall'Università di Genova) può essere scaricato dal repository UCI (https://archive.ics.uci.edu/ml/datasets/wine). 
Contiene 178 osservazioni, ognuna corrispondente ad una bottiglia di vino: 
* Le feature (`x`): un vettore 13-dimensionale consistente di caratteristiche chimiche e visuali della bottiglia di vino
* Le etichette (`y`): la cantina di provenienza della bottiglia (1,2,3)

Prima di continuare, assicurarsi che il dataset (`wine.data.txt`) sia presente in una sottocartella di nome `data`.

## 1. Caricamento del dataset

Iniziamo caricando i pacchetti Python che useremo. 

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Un modulo utile per la gestione di distribuzioni gaussiane
from scipy.stats import norm, multivariate_normal
# Moduli per la visualizzazione di grafici interattivi
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

Ora carichiamo il dataset Wine. Ci sono 178 osservazioni, ciascuna con 13 feature ed un'etichetta (1,2,3). 
Le divideremo in un training set di 130 punti ed un test set di 48 punti. 

In [4]:
data = np.loadtxt('data/wine.data.txt', delimiter=',')
# Nomi delle feature
featurenames = ['Alcool', 'Acido malico', 'Ceneri', 'Alcalinità delle ceneri', 'Magnesio', 'Fenoli totali', 
               'Flavonoidi', 'Fenoli non flavonoidi', 'Proantocianina', 'Intensità di colore', 'Tonalità',
              'OD280/OD315', 'Prolina']

Fissiamo una particulare permutazione pseudocasuale dei dati, ed usiamola per effettuare la partizione in training e test.
Vogliamo creare 4 array:
* `trainx`: 130x13, le osservazioni di training
* `trainy`: 130x1, le etichette delle osservazioni di training
* `testx`: 48x13, le osservazioni di test
* `testy`: 48x1, le etichette delle osservazioni di test

In [5]:
# Suddividiamo le 178 istanze in un training set (trainx, trainy) di taglia 130 e un test set (testx, testy) di taglia 48
# Separiamo inoltre i dati di input (colonne 1-13) dalle etichette (colonna 0)
np.random.seed(0) # inizializzazione del generatore pseudo-casuale (per la riproducibilità degli esperimenti)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

Verifichiamo quanti dati di training ci sono in ogni classe. 

In [6]:
sum(trainy==1), sum(trainy==2), sum(trainy==3)

(43, 54, 33)

### Esercizio rapido 1

Quante istanze di test sono presenti in ciascuna classe?

In [47]:
sum(testy==1), sum(testy==2), sum(testy==3)

(16, 17, 15)

In [48]:
# modificate questa cella
for x in range(1,4):
    print(len([x for x in testy[[testy==x]]]))
 



16
17
15


C:\Users\finnocenti\AppData\Local\Temp\ipykernel_15688\2342593116.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  print(len([x for x in testy[[testy==x]]]))


Cliccare **qui** per la soluzione. 
<!--
sum(testy==1), sum(testy==2), sum(testy==3)
-->

## 2. Costruzione della distribuzione di una singola feature da ognuna delle cantine

Focalizziamoci su una singola feature: 'Alcool'. Questa è la prima feature, quella di indice 0. Il seguente grafico è un'*istogramma* dei valori di questa feature nella classe 1 (cantina 1), insieme al *fit gaussiano* della distribuzione. 

<img src="img/wine_histogram.png">


Come si può generare un grafico di questo tipo? 

La seguente funzione, **density_plot**, lo fa per ogni feature ed etichetta. La prima riga aggiunge una componente interattiva che ci permette di scegliere questi parametri utilizzando dei controlli. 

Provatelo, e poi, osservate il codice attentamente per capire esattamente ciò che fa ciascuna riga della funzione.

In [49]:
@interact_manual( feature=IntSlider(0,0,12), label=IntSlider(1,1,3))
def density_plot(feature, label):
    plt.hist(trainx[trainy==label,feature], density=True)
    #
    mu = np.mean(trainx[trainy==label,feature]) # media
    var = np.var(trainx[trainy==label,feature]) # varianza
    std = np.sqrt(var) # deviazione standard
    #
    x_axis = np.linspace(mu - 3*std, mu + 3*std, 1000)
    plt.plot(x_axis, norm.pdf(x_axis,mu,std), 'r', lw=2)
    plt.title("Cantina "+str(label) )
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Densità', fontsize=14, color='red')
    plt.show()

interactive(children=(IntSlider(value=0, description='feature', max=12), IntSlider(value=1, description='label…

### Esercizio rapido 2

Domanda: per quale feature (indice da 0 a 12) la distribuzione dei valori (nel training set) della Cantina 1 ha la deviazione standard *minima*? 

In [61]:
std = np.zeros(13)
for feature in range(0,13):
    std[feature] = np.sqrt(np.var(trainx[trainy==1,feature]))
print(std)
np.argmin(std)

[4.82962509e-01 6.56756786e-01 1.91767278e-01 2.45766535e+00
 1.08840191e+01 3.43734147e-01 3.90396479e-01 5.96428889e-02
 4.53274368e-01 1.22463376e+00 1.15433202e-01 3.55846328e-01
 2.20103973e+02]


7

In [79]:
import pandas as pd
import numpy as np
import time

In [95]:
t_before = time.time()
std=np.zeros(13)
for i in range(0,13):
    std[i]=np.sqrt([np.var(trainx[trainy==1,i])])
print(np.argmin(std))
t_after = time.time()
print("Tempo (secondi): ", t_after - t_before)

7
Tempo (secondi):  0.0038461685180664062


In [93]:
t_before = time.time()
std=pd.DataFrame()
for i in range(0,13):
    std[i]=np.sqrt([np.var(trainx[trainy==1,i])])
print(np.argmin(std))
t_after = time.time()
print("Tempo (secondi): ", t_after - t_before)


7
Tempo (secondi):  0.018377304077148438


In [92]:
# modificate questa cella
t_before = time.time()
var=[]
for x in range(0,13):
    var_temp= np.sqrt([np.var(trainx[trainy==1,x])])
    var.append(var_temp)
    
print(np.argmin(var))
t_after = time.time()
print("Tempo (secondi): ", t_after - t_before)


7
Tempo (secondi):  0.0020275115966796875


Cliccare **qui** per la soluzione. 
<!--
std = np.zeros(13)
for feature in range(0,13):
    std[feature] = np.sqrt(np.var(trainx[trainy==1,feature]))
print(std)
np.argmin(std)
-->

## 3. Fit di una gaussiana per ogni classe

Definiamo una funzione che effettua il fit di un modello generativo gaussiano alle tre classi, restringendosi ad un'unica feature. 

In [ ]:
# La funzione assume che l'etichetta y vari nell'insieme {1,2,3}
def fit_generative_model(x,y,feature):
    k = 3 # numero di classi
    mu = np.zeros(k+1) # lista delle medie
    var = np.zeros(k+1) # lista delle varianze
    pi = np.zeros(k+1) # vettore dei pesi delle classi
    for label in range(1,k+1):
        indices = (y==label)
        mu[label] = np.mean(x[indices,feature])
        var[label] = np.var(x[indices,feature])
        pi[label] = float(sum(indices))/float(len(y))
    return mu, var, pi

Chiamiamo questa funzione sulla feature di indice 0 (Alcool). Quali sono i pesi delle classi? 

In [ ]:
feature = 0 # 'alcool'
mu, var, pi = fit_generative_model(trainx, trainy, feature)
print(pi[1:])
print(mu[1:])
print(var[1:])

Grafichiamo la distribuzione Gaussiana risultante per ciascuna delle tre classi.

In [ ]:
@interact_manual( feature=IntSlider(0,0,12) )
def show_densities(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    colors = ['r', 'k', 'g']
    for label in range(1,4):
        m = mu[label]
        s = np.sqrt(var[label])
        x_axis = np.linspace(m - 3*s, m+3*s, 1000)
        plt.plot(x_axis, norm.pdf(x_axis,m,s), colors[label-1], label="Cantina " + str(label))
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Densità', fontsize=14, color='red')
    plt.legend()
    plt.show()

### Esercizio rapido 3

Usando il grafico interattivo soprastante, rispondere alle seguenti domande: 
* Per quale feature (0-12) le distribuzioni della classe 1 e 3 si *sovrappongono* di più? 
* Per quale feature (0-12) la classe 3 è quella con maggiore varianza rispetto alle altre due classi? 
* Per quale feature (0-12) le tre classi sembrano meglio *separate* (almeno ad occhio)? 

Cliccare **qui** per le risposte. 
<!--
* Feature 2 (Ceneri)
* Feature 9 (Intensità di colore)
* Feature 6 (Flavonoidi)
-->

## 4. Predizione delle etichette sul test set

Quanto accuratamente possiamo predire la classe (1,2,3) sulla base di una singola feature? Il codice sottostante ci consente di scoprirlo. 

In [ ]:
@interact( feature=IntSlider(0,0,12) )
def test_model(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    k = 3 # Etichette 1,2,...,k
    n_test = len(testy) # Numero di osservazioni di test
    score = np.zeros((n_test,k+1))
    for i in range(0,n_test):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            norm.logpdf(testx[i,feature], mu[label], np.sqrt(var[label]))
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    # Conteggiamo il numero di predizioni errate
    errors = np.sum(predictions != testy)
    print("Errori di test sulla base della feature " + featurenames[feature] + ": " + str(errors) + "/" + str(n_test))

### Esercizio 4

In questo quaderno, abbiamo considerato classificatori che usano solo una delle 13 possibili feature. La scelta di un sottoinsieme di feature è detta **feature selection**. In generale, questa scelta va operata sulla base del *training set* (o meglio, sulla base di un apposito *validation set*) -- in particolare, senza considerare il *test set*. 

Per i dati sul vino, calcolate l'errore di training e l'errore di test associato con ogni possibile scelta di una feature. 

In [ ]:
### Scrivete il vostro codice qui

Sulla base dei vostri risultati, rispondere alle seguenti domande: 
* Quali tre feature hanno l'errore più basso sul training set? 
* Quali tre feature hanno l'errore più basso sul test set? 